In [1]:
import uvicorn
from fastapi import FastAPI
import numpy as np
import pandas as pd
import pickle
from pydantic import BaseModel

In [2]:
description = """
The Flight fare prediction API helps you to determine the price of your next flight with just a few simple inputs

# Items

You can **predict** Flight fare rates

# Users

Just enter the fields below in the predict tab to make predictions

All values are to be input as integers

**Total_stops**: Number of stops required(Max 4)

**Date**: Date of travel

**Month**: Month of travel

**Dep_hour**: Hour of yor departure(hour can be input in 24 hour format)

**Dep_min**: Minute of your departure

For the values below, Please input your preferred choice as per the keys shown below

**Airline**: The airline you wish to travel by ('Air Asia': 0, 'Air India': 1, 'GoAir': 2, 'IndiGo': 3, 'Jet Airways': 4, 'Jet Airways Business': 5, 'Multiple carriers': 6, 'Multiple carriers Premium economy': 7, 'SpiceJet': 8, 'Vistara': 9, 'Vistara Premium economy': 10)

**Source(From)**: Your port of departure ('Banglore': 0, 'Chennai': 1, 'Delhi': 2, 'Kolkata': 3, 'Mumbai': 4)

**Destination(To)**: Your port of arrival ('Banglore': 0, 'Cochin': 1, 'Delhi': 2, 'Hyderabad': 3, 'Kolkata': 4)

"""

In [3]:
# Initializing the app object
app = FastAPI(title= "Flight fare", description = description)

In [4]:
# Opening and loading pickled files
pickle_in = open('rf_reg_model2.pkl',"rb")
model = pickle.load(pickle_in)

In [5]:
@app.get('/')
def index():
    return {'Message': 'Welcome to Flight Fare prediction. Type /docs at the end of the above url'}


In [6]:
class fare(BaseModel):
    Total_Stops: int
    Date: int
    Month: int
    Dep_hour: int
    Dep_min: int
    Airline: int
    Source: int
    Destination: int

In [7]:
@app.post('/predict')
def predict(data: fare):
    data = data.dict()
    Total_Stops = data['Total_Stops']
    Date = data['Date']
    Month = data['Month']
    Dep_hour = data['Dep_hour']
    Dep_min = data['Dep_min']
    Airline = data['Airline']
    Source = data['Source']
    Destination = data['Destination']


    prediction=model.predict([[
        Total_Stops,
        Date,
        Month,
        Dep_hour,
        Dep_min,
        Airline,
        Source,
        Destination]])
    
    print(np.array(list(data.values())).reshape(1,-1))

    output=round(prediction[0],2)
    return {"The Flight fare is Rs. {}".format(output)}

In [8]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
if __name__ == '__main__':
    uvicorn.run(app, host = '127.0.0.1', port = 8000)

INFO:     Started server process [10312]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51758 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:51759 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51759 - "GET /openapi.json HTTP/1.1" 200 OK
[[ 1  6  6 17 30  4  2  1]]
INFO:     127.0.0.1:51765 - "POST /predict HTTP/1.1" 200 OK
[[ 0 24  6 23 55  0  0  2]]
INFO:     127.0.0.1:51767 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [10312]
